In [22]:
## Imports
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
import gc
import pickle

In [23]:
def preprocess_image(file_path, label):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_png(image, channels=1)  # MURA images are grayscale
    image = tf.image.resize(image, [224, 224])      # Match ResNet/VGG input shape
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


In [24]:
with open('mura_train.pkl', 'rb') as f:
    train_images, train_labels = pickle.load(f)

with open('mura_val.pkl', 'rb') as f:
    val_images, val_labels = pickle.load(f)

# Rebuild datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_ds = train_ds.map(preprocess_image).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_ds = val_ds.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)


In [25]:
# Clear previous session
tf.keras.backend.clear_session()
gc.collect()

# Build custom CNN
inputs = Input(shape=(224, 224, 1))

x = Conv2D(32, (3, 3), padding='same', activation='relu',
           kernel_initializer='glorot_uniform')(inputs)
x = BatchNormalization()(x)
x = Conv2D(32, (3, 3), padding='same', activation='relu',
           kernel_initializer='glorot_uniform')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.25)(x)

x = Conv2D(64, (3, 3), padding='same', activation='relu',
           kernel_initializer='glorot_uniform')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), padding='same', activation='relu',
           kernel_initializer='glorot_uniform')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.25)(x)

x = Conv2D(128, (3, 3), padding='same', activation='relu',
           kernel_initializer='glorot_uniform')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.4)(x)

x = Flatten()(x)
x = Dense(256, activation='relu', kernel_initializer='glorot_uniform')(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid', kernel_initializer='glorot_uniform')(x)

model = Model(inputs=inputs, outputs=outputs)


In [ ]:
model.summary()

In [ ]:
# Compile
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[early_stopping]
)


Epoch 1/20
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 4048s 4s/step - accuracy: 0.7991 - loss: 0.6893 - val_accuracy: 0.5224 - val_loss: 3.1020
Epoch 2/20
 879/1151 ━━━━━━━━━━━━━━━━━━━━ 20:35 5s/step - accuracy: 0.7263 - loss: 0.6028

In [ ]:
# Evaluate
test_loss, test_acc = model.evaluate(val_ds)
print("Validation accuracy:", test_acc)

# Accuracy plot
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()

# Loss plot
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Predict
y_true = []
y_pred = []

for images, labels in val_ds:
    preds = model.predict(images)
    y_pred.extend((preds > 0.5).astype(int).flatten())
    y_true.extend(labels.numpy())

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)

plt.imshow(cm, cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.colorbar()
plt.show()
